In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [90]:
from sklearn.preprocessing import StandardScaler, scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, RandomForestRegressor

# PIMA Indians Dataset

In [3]:
pima = pd.read_csv('./pima.csv')
print(pima.shape)
pima.head()

(392, 9)


,pregnant,glucose,diastolic,triceps,insulin,bmi,family,age,test
0,1,89,66,23,94,28.1,0.167,21,negative
1,0,137,40,35,168,43.1,2.288,33,positive
2,3,78,50,32,88,31.0,0.248,26,positive
3,2,197,70,45,543,30.5,0.158,53,positive
4,1,189,60,23,846,30.1,0.398,59,positive


In [4]:
pima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pregnant   392 non-null    int64  
 1   glucose    392 non-null    int64  
 2   diastolic  392 non-null    int64  
 3   triceps    392 non-null    int64  
 4   insulin    392 non-null    int64  
 5   bmi        392 non-null    float64
 6   family     392 non-null    float64
 7   age        392 non-null    int64  
 8   test       392 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 27.7+ KB


In [16]:
X = pima.drop('test', axis=1)
y = pima['test']

## Logistic Regression

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

In [18]:
# initialize scaler and fit
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [19]:
# scale data
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [9]:
# initialize logreg and fit
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

# predict
y_pred = lr.predict(X_test_std)

In [10]:
# eval
print(f'{accuracy_score(y_test, y_pred): .1%} test acc')
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

 80.6% test acc
{'pregnant': 0.18, 'glucose': 1.19, 'diastolic': 0.05, 'triceps': 0.15, 'insulin': 0.17, 'bmi': 0.45, 'family': 0.29, 'age': 0.44}


## Manual Recursive Feature Elimination

Here, we train the model and remove the coefficients with lowest values repeatedly and see if we do so without hurting the model accuracy too much.

In [11]:
# In the previous training, 'diastolic' had the lowest value so we remove that
X = pima.drop(['test', 'diastolic'], axis=1)
y = pima['test']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

# initialize scaler and fit
scaler = StandardScaler()
scaler.fit(X_train)

# scale data
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

# initialize logreg and fit
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

# predict
y_pred = lr.predict(X_test_std)

# eval
print(f'{accuracy_score(y_test, y_pred): .1%} test acc')
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

 80.6% test acc
{'pregnant': 0.18, 'glucose': 1.18, 'triceps': 0.15, 'insulin': 0.17, 'bmi': 0.43, 'family': 0.29, 'age': 0.42}


In [12]:
# Then we take away the next 2 lowest: 'insulin' and 'triceps'
X = pima.drop(['test', 'diastolic', 'insulin', 'triceps'], axis=1)
y = pima['test']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

# initialize scaler and fit
scaler = StandardScaler()
scaler.fit(X_train)

# scale data
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

# initialize logreg and fit
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

# predict
y_pred = lr.predict(X_test_std)

# eval
print(f'{accuracy_score(y_test, y_pred): .1%} test acc')
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

 81.6% test acc
{'pregnant': 0.18, 'glucose': 1.09, 'bmi': 0.5, 'family': 0.3, 'age': 0.43}


In [13]:
# Lastly, we only keep the highest scoring feature
X = pima['glucose'].to_numpy().reshape(-1, 1)
y = pima['test']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

# initialize scaler and fit
scaler = StandardScaler()
scaler.fit(X_train)

# scale data
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

# initialize logreg and fit
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

# predict
y_pred = lr.predict(X_test_std)

# eval
print(f'{accuracy_score(y_test, y_pred): .1%} test acc')
print(dict(zip(['glucose'], abs(lr.coef_[0]).round(2))))

 76.5% test acc
{'glucose': 1.28}


## Automatic Recursive Feature Elimination using `RFE` from `sklearn`

In [ ]:
X = pima.drop('test', axis=1)
y = pima['test']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

# initialize scaler and fit
scaler = StandardScaler()
scaler.fit(X_train)

# scale data
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [20]:
# initialize rfe
rfe = RFE(estimator=LogisticRegression(),
          n_features_to_select=3,
          verbose=1)

In [21]:
rfe.fit(X_train_std, y_train)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.


RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

In [22]:
# print features and their ranking (high=dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

{'pregnant': 4, 'glucose': 1, 'diastolic': 6, 'triceps': 5, 'insulin': 3, 'bmi': 1, 'family': 2, 'age': 1}


In [23]:
# print features that are not eliminated
print(X.columns[rfe.support_])

Index(['glucose', 'bmi', 'age'], dtype='object')


In [24]:
# eval
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print(f'test acc: {acc: .1%}')

test acc:  79.6%


## Random Forest Classifier

In [25]:
# initialize rf and fit
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [26]:
# eval
acc = accuracy_score(y_test, rf.predict(X_test))
print(f'test acc: {acc: .1%}')

# feature importances
print(dict(zip(X.columns, rf.feature_importances_.round(2))))

test acc:  80.6%
{'pregnant': 0.08, 'glucose': 0.22, 'diastolic': 0.07, 'triceps': 0.09, 'insulin': 0.15, 'bmi': 0.12, 'family': 0.11, 'age': 0.16}


## Random forest for feature selection

In [27]:
# create mask for feature importances above threshold
mask = rf.feature_importances_ > 0.15

In [28]:
# apply mask to X
X_reduced = X.iloc[:, mask]
X_reduced.columns

Index(['glucose', 'age'], dtype='object')

## `RFE` with random forests

In [29]:
# initialize RFE with rf with step=1
rfe = RFE(estimator=RandomForestClassifier(random_state=0),
          n_features_to_select=2,
          verbose=1)

rfe.fit(X_train, y_train)

mask = rfe.support_

X_reduced = X.loc[:, mask]
print(X_reduced.columns)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Index(['glucose', 'insulin'], dtype='object')


In [30]:
# initialize RFE with rf with step=2
rfe = RFE(estimator=RandomForestClassifier(random_state=0),
          n_features_to_select=2,
          step=2,
          verbose=1)

rfe.fit(X_train, y_train)

mask = rfe.support_

X_reduced = X.loc[:, mask]
print(X_reduced.columns)

Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 4 features.
Index(['glucose', 'age'], dtype='object')


# Male ANSUR Measurements Dataset

In [32]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [33]:
ansur_m = pd.read_csv('./ANSUR_II_MALE.csv')
print(ansur_m.shape)
ansur_m.head(3)

(4082, 99)


,Branch,Component,Gender,abdominalextensiondepthsitting,acromialheight,acromionradialelength,anklecircumference,axillaheight,balloffootcircumference,balloffootlength,biacromialbreadth,bicepscircumferenceflexed,bicristalbreadth,bideltoidbreadth,bimalleolarbreadth,bitragionchinarc,bitragionsubmandibulararc,bizygomaticbreadth,buttockcircumference,buttockdepth,buttockheight,buttockkneelength,buttockpopliteallength,calfcircumference,cervicaleheight,chestbreadth,chestcircumference,chestdepth,chestheight,crotchheight,crotchlengthomphalion,crotchlengthposterioromphalion,earbreadth,earlength,earprotrusion,elbowrestheight,eyeheightsitting,footbreadthhorizontal,footlength,forearmcenterofgriplength,forearmcircumferenceflexed,forearmforearmbreadth,forearmhandlength,functionalleglength,handbreadth,handcircumference,handlength,headbreadth,headcircumference,headlength,heelanklecircumference,heelbreadth,hipbreadth,hipbreadthsitting,iliocristaleheight,interpupillarybreadth,interscyei,interscyeii,kneeheightmidpatella,kneeheightsitting,lateralfemoralepicondyleheight,lateralmalleolusheight,lowerthighcircumference,mentonsellionlength,neckcircumference,neckcircumferencebase,overheadfingertipreachsitting,palmlength,poplitealheight,radialestylionlength,shouldercircumference,shoulderelbowlength,shoulderlength,sittingheight,sleevelengthspinewrist,sleeveoutseam,span,suprasternaleheight,tenthribheight,thighcircumference,thighclearance,thumbtipreach,tibialheight,tragiontopofhead,trochanterionheight,verticaltrunkcircumferenceusa,waistbacklength,waistbreadth,waistcircumference,waistdepth,waistfrontlengthsitting,waistheightomphalion,wristcircumference,wristheight,weight_kg,stature_m,BMI,BMI_class,Height_class
0,Combat Arms,Regular Army,Male,266,1467,337,222,1347,253,202,401,369,274,493,71,319,291,142,979,240,882,619,509,373,1535,291,1074,259,1292,877,607,351,36,71,19,247,802,101,273,349,299,575,477,1136,90,214,193,150,583,206,326,70,332,366,1071,685,422,441,502,560,500,77,391,118,400,436,1447,113,437,273,1151,368,145,928,883,600,1782,1449,1092,610,164,786,491,140,919,1700,501,329,933,240,440,1054,175,853,81.5,1.776,25.838761,Overweight,Tall
1,Combat Support,Regular Army,Male,233,1395,326,220,1293,245,193,394,338,257,479,67,344,320,135,944,232,870,584,468,357,1471,269,1021,253,1244,851,615,376,33,62,18,232,781,98,263,348,289,523,476,1096,86,203,195,146,568,201,334,72,312,356,1046,620,441,447,490,540,488,73,371,131,380,420,1380,118,417,254,1119,353,141,884,868,564,1745,1387,1076,572,169,822,476,120,918,1627,432,316,870,225,371,1054,167,815,72.6,1.702,25.062103,Overweight,Normal
2,Combat Support,Regular Army,Male,287,1430,341,230,1327,256,196,427,408,261,544,75,345,330,135,1054,258,901,623,506,412,1501,288,1120,267,1288,854,636,359,40,61,23,237,810,103,270,355,357,575,491,1115,93,220,203,148,573,202,356,70,349,393,1053,665,462,475,496,556,482,72,409,123,403,434,1447,121,431,268,1276,367,167,917,910,604,1867,1438,1105,685,198,807,477,125,918,1678,472,329,964,255,411,1041,180,831,92.9,1.735,30.861480,Overweight,Normal


## LASSO Regressor

In [37]:
X = ansur_m.iloc[:, 3:94] # here, we work only on the numerical measurements
y = ansur_m['BMI'] # we try to predict the BMI

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=0)

In [51]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [54]:
# initialize Lasso regressor
la = Lasso()
# fit
la.fit(X_train_std, y_train)
# eval
print(f'The model can predict {la.score(X_test_std, y_test): .1%} of the variance in the test seet.')

The model can predict  84.7% of the variance in the test seet.


In [55]:
# create a list that has True values when coefs are 0
zero_coef = la.coef_ == 0
# calculate how many features have a zero coefficient
print(f'The model has ignored {sum(zero_coef)} out of {len(la.coef_)} features.')

The model has ignored 82 out of 91 features.


## Adjusting the regularization strength

When a model applies overly powerful regularization, it can suffer from high bias, so we try to improve by tweaking `alpha` and find which can predict above 98\% while ignoring more features.

In [57]:
param_grid = {'alpha': [0.01, 0.1, 0.5, 1]}

la = Lasso(random_state=0)

la_grid = GridSearchCV(estimator=la,
                       param_grid=param_grid,
                       cv=5)

la_grid.fit(X_train_std, y_train)

GridSearchCV(cv=5, estimator=Lasso(random_state=0),
             param_grid={'alpha': [0.01, 0.1, 0.5, 1]})

In [61]:
print(la_grid.best_params_)

la_best = la_grid.best_estimator_
print(f'The model can predict {la_best.score(X_test_std, y_test): .1%} of the variance in the test set.')

print(f'{sum(la_best.coef_ == 0)} out of {len(la_best.coef_)} features were ignored')

{'alpha': 0.01}
The model can predict  98.8% of the variance in the test set.
37 out of 91 features were ignored


However, only 37 features were ignored. Let's check the results if there are others that achieved above 98\%. Seems `alpha=0.1` may work too.

In [65]:
pd.DataFrame(la_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.168481,0.014205,0.000400,0.000490,0.01,{'alpha': 0.01},0.987437,0.987411,0.987564,0.988660,0.985895,0.987393,0.000881,1
1,0.056252,0.007660,0.003122,0.006243,0.1,{'alpha': 0.1},0.982062,0.981131,0.982023,0.982882,0.980311,0.981682,0.000881,2
2,0.027299,0.012683,0.000600,0.000800,0.5,{'alpha': 0.5},0.935583,0.932203,0.930501,0.930899,0.934035,0.932644,0.001919,3
3,0.028005,0.006193,0.000400,0.000799,1,{'alpha': 1},0.843456,0.836537,0.833907,0.836136,0.840540,0.838115,0.003421,4


In [66]:
la = Lasso(alpha=0.1, random_state=0)

la.fit(X_train_std, y_train)

print(f'The model can predict {la.score(X_test_std, y_test): .1%} of the variance in the test set.')
print(f'{sum(la.coef_ == 0)} out of {len(la.coef_)} features were ignored')

The model can predict  98.3% of the variance in the test set.
64 out of 91 features were ignored


## Combining feature selectors: `LassoCV` Regressor

Here, we'll be working with a subsample of the Male ANSUR and try to predict biceps circumference.

In [67]:
subsample = ['acromialheight', 'axillaheight', 'bideltoidbreadth', 'buttockcircumference', 'buttockkneelength', 'buttockpopliteallength', 'cervicaleheight', 'chestcircumference', 'chestheight',
             'earprotrusion', 'footbreadthhorizontal', 'forearmcircumferenceflexed', 'handlength', 'headbreadth', 'heelbreadth', 'hipbreadth', 'iliocristaleheight', 'interscyeii',
             'lateralfemoralepicondyleheight', 'lateralmalleolusheight', 'neckcircumferencebase', 'radialestylionlength', 'shouldercircumference', 'shoulderelbowlength', 'sleeveoutseam',
             'thighcircumference', 'thighclearance', 'verticaltrunkcircumferenceusa', 'waistcircumference', 'waistdepth', 'wristheight', 'BMI']

In [71]:
X = ansur_m[subsample]
y = ansur_m['bicepscircumferenceflexed']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=0)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [74]:
# initialize LassoCV and fit
lcv = LassoCV()
lcv.fit(X_train_std, y_train)
print(f'Optimal alpha {lcv.alpha_: .3f}')

print(f'The model explains {lcv.score(X_test_std, y_test): .1%} of the test set variance')

#create mask for coefs not equal to zero
lcv_mask = lcv.coef_ != 0
print(f'{sum(lcv_mask)} features out of {len(lcv_mask)} selected.')

Optimal alpha  0.035
The model explains  85.6% of the test set variance
24 features out of 32 selected.


## Combining feature selectors: Ensemble models for extra votes

In [79]:
# Select 10 features with RFE on Gradient Boosting Regressor
# drop 3 on each step
rfe_gb = RFE(estimator=GradientBoostingRegressor(),
             n_features_to_select=10,
             step=3,
             verbose=1)
rfe_gb.fit(X_train_std, y_train)

print(f'The model can explain {rfe_gb.score(X_test_std, y_test): .1%} of the variance in the test set')

# assign the support array to gb_mask
gb_mask = rfe_gb.support_

Fitting estimator with 32 features.
Fitting estimator with 29 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
The model can explain  83.3% of the variance in the test set


In [81]:
# Do the same for a Random Forest Regressor
rfe_rf = RFE(estimator=RandomForestRegressor(),
             n_features_to_select=10,
             step=3,
             verbose=1)
rfe_rf.fit(X_train_std, y_train)

print(f'The model can explain {rfe_rf.score(X_test_std, y_test): .1%} of the variance in the test set')

# assign the support array to gb_mask
rf_mask = rfe_rf.support_

Fitting estimator with 32 features.
Fitting estimator with 29 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
The model can explain  82.4% of the variance in the test set


## Combining the 3 feature selectors

In [89]:
# sum votes of 3 models
votes = np.sum([lcv_mask, rf_mask, gb_mask], axis=0)

# create mask for features selected by all 3 models
meta_mask = votes >= 3

# apply the dim reduction on X
X_reduced = X.loc[:, meta_mask]
X_reduced.columns

Index(['bideltoidbreadth', 'buttockcircumference', 'chestcircumference',
       'forearmcircumferenceflexed', 'shouldercircumference',
       'shoulderelbowlength', 'thighcircumference', 'BMI'],
      dtype='object')

In [91]:
# Plug the reduced dataset into a linear regression
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y,
                                                    test_size=0.3,
                                                    random_state=0)
scaler = StandardScaler()
lm = LinearRegression()

lm.fit(scaler.fit_transform(X_train), y_train)

r_2 = lm.score(scaler.transform(X_test), y_test)
print(f'The model can explain {r_2: .1%} of the variance in the test set using {len(lm.coef_)} features.')

The model can explain  84.0% of the variance in the test set using 8 features.


Using the votes from 3 models, just 7 features were selected to allow a simple linear model to get a high accuracy.